In [115]:
#r "nuget: Microsoft.Azure.Storage.Common"
#r "nuget: Microsoft.Azure.Storage.Blob"
#r "nuget: Microsoft.Azure.Storage.File"

In [116]:
using Microsoft.Azure.Storage;
using Microsoft.Azure.Storage.Blob;
using Microsoft.Azure.Storage.DataMovement;
using Microsoft.Azure.Storage.File;
using System.IO;
using System.IO.Compression;
using Azure.Identity;

using CopyStatus = Microsoft.Azure.Storage.Blob.CopyStatus;
using Microsoft.Spark.Extensions.Azure.Synapse.Analytics.Utils;
using Microsoft.Azure.Storage.Auth;


In [117]:
CloudBlobContainer destContainer, sourceContainer;
CloudBlobDirectory destDirectory;

string destConnectionString = "";

In [118]:
private async Task InitSourceAndDest ()
{
    // Create a BlobEndpoint for the source container
    string sourceConnectionString = "BlobEndpoint=https://lilablobssc.blob.core.windows.net;";
    CloudStorageAccount sourceStorageAccount = CloudStorageAccount.Parse(sourceConnectionString);
    CloudBlobClient sourceBlobClient = sourceStorageAccount.CreateCloudBlobClient();
    sourceContainer = sourceBlobClient.GetContainerReference("snapshotserengeti-v-2-0");

    // Create a FileEndpoint for the destination ADLS
    CloudStorageAccount destStorageAccount = CloudStorageAccount.Parse(destConnectionString);

    var destBlobClient= destStorageAccount.CreateCloudBlobClient();
    destContainer =  destBlobClient.GetContainerReference("snapshot-serengeti");

    if(! await destContainer.ExistsAsync())
    {
        await destContainer.CreateIfNotExistsAsync();
    }

    destDirectory = destContainer.GetDirectoryReference("metadata");
}

In [119]:
private async Task ExtractAndSaveZippedBlob(CloudBlockBlob sourceBlob,   CloudBlobDirectory destDirectory)
{
    using (var sourceStream = sourceBlob.OpenRead())
    using (var archive = new ZipArchive(sourceStream))
    {
        foreach (var entry in archive.Entries)
        {
            var destBlob = destDirectory.GetBlockBlobReference(entry.FullName);

            if (await destBlob.ExistsAsync())
            {
                Console.WriteLine($"{destBlob.Name} exists");
                continue;
            }


            using (var entryStream = entry.Open())
            {
                Console.WriteLine($"Uploading {destBlob.Name}");
                await destBlob.UploadFromStreamAsync(entryStream);
            }
        }
    }
}

In [120]:
private async Task CopyBlob(CloudBlockBlob sourceBlob, CloudBlockBlob destBlob)
{
    if (await destBlob.ExistsAsync())
    {
        Console.WriteLine($"{destBlob.Name} exists");
        return;
    }

    await destBlob.StartCopyAsync(sourceBlob.Uri);

    while (destBlob.CopyState.Status == CopyStatus.Pending)
    {
        await Task.Delay(1000); //wait for 1 second before checking again
        await destBlob.FetchAttributesAsync();
    }

    if (destBlob.CopyState.Status == CopyStatus.Success)
    {
        Console.WriteLine("Copy operation succeeded");
    }
    else
    {
        Console.WriteLine("Copy operation failed");
    }
}


In [121]:
// s01 to s011
var metadataZippedFiles=new string[]
{
    "SnapshotSerengetiS01.json.zip",
    "SnapshotSerengetiS02.json.zip",
    "SnapshotSerengetiS03.json.zip",
    "SnapshotSerengetiS04.json.zip",
    "SnapshotSerengetiS05.json.zip",
    "SnapshotSerengetiS06.json.zip",
    "SnapshotSerengetiS07.json.zip",
    "SnapshotSerengetiS08.json.zip",
    "SnapshotSerengetiS09.json.zip",
    "SnapshotSerengetiS10.json.zip",
    "SnapshotSerengetiS11.json.zip",
    "SnapshotSerengetiBboxes_20190903.json.zip",

};

await InitSourceAndDest();

foreach (var zippedFile in metadataZippedFiles)
{
    var sourceBlob = sourceContainer.GetBlockBlobReference(zippedFile);
    await ExtractAndSaveZippedBlob(sourceBlob, destDirectory);
}

var jsonFiles = new string[]
{
        "SnapshotSerengetiSplits_v0.json"
};

foreach (var file in jsonFiles)
{
    var sourceBlob = sourceContainer.GetBlockBlobReference(file);
    var destBlob = destDirectory.GetBlockBlobReference(file);
    await CopyBlob(sourceBlob, destBlob);
}
